In [7]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# using tensorflow>=2.2.1 with built-in Keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, PReLU
import tensorflow.keras.utils as np_utils

In [8]:
TEST_DATA_ROOT = '~/workspace/datarobot-user-models/tests/testdata'
BINARY_DATA = os.path.join(
    TEST_DATA_ROOT, 'iris_binary_training.csv'
)
REGRESSION_DATA = os.path.join(
    TEST_DATA_ROOT, 'boston_housing.csv'
)
MULTICLASS_DATA = os.path.join(
    TEST_DATA_ROOT, 'Skyserver_SQL2_27_2018 6_51_39 PM.csv'
)

bin_X = pd.read_csv(BINARY_DATA)
bin_y = bin_X.pop('Species')

reg_X = pd.read_csv(REGRESSION_DATA)
reg_y = reg_X.pop('MEDV')

multi_X = pd.read_csv(MULTICLASS_DATA)
multi_y = multi_X.pop('class')

In [9]:
bin_model = Sequential([
    Dense(50, input_dim=bin_X.shape[1]),
    Activation('relu'),
    Dropout(0.2),
    Dense(100),
    PReLU(),
    Dense(1),
    Activation('sigmoid')
])
bin_model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
bin_target_encoder = LabelEncoder()
bin_target_encoder.fit(bin_y)
bin_model.fit(bin_X, bin_target_encoder.transform(bin_y), epochs=10, batch_size=32)

reg_model = Sequential([
    Dense(50, input_dim=reg_X.shape[1]),
    Activation('relu'),
    Dropout(0.2),
    Dense(100),
    PReLU(),
    Dense(1),
])
reg_model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])
reg_model.fit(reg_X, reg_y, epochs=10, batch_size=32)

multi_target_encoder = LabelEncoder()
multi_target_encoder.fit(multi_y)
multi_model = Sequential()
multi_model.add(Dense(8, input_dim=multi_X.shape[1], activation='relu'))
multi_model.add(Dense(len(multi_target_encoder.classes_), activation='softmax'))
multi_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
multi_model.fit(multi_X, np_utils.to_categorical(multi_target_encoder.transform(multi_y)), epochs=10, batch_size=32)

Train on 109 samples
Epoch 1/10
109/109 [==============================] - 0s 850us/sample - loss: 1.5773 - acc: 0.3670
Epoch 2/10
109/109 [==============================] - 0s 78us/sample - loss: 0.9270 - acc: 0.3945
Epoch 3/10
109/109 [==============================] - 0s 66us/sample - loss: 1.0007 - acc: 0.5688
Epoch 4/10
109/109 [==============================] - 0s 68us/sample - loss: 0.6493 - acc: 0.7156
Epoch 5/10
109/109 [==============================] - 0s 83us/sample - loss: 0.5920 - acc: 0.6972
Epoch 6/10
109/109 [==============================] - 0s 56us/sample - loss: 0.5241 - acc: 0.7798
Epoch 7/10
109/109 [==============================] - 0s 53us/sample - loss: 0.5650 - acc: 0.7890
Epoch 8/10
109/109 [==============================] - 0s 52us/sample - loss: 0.4209 - acc: 0.8165
Epoch 9/10
109/109 [==============================] - 0s 60us/sample - loss: 0.3901 - acc: 0.8624
Epoch 10/10
109/109 [==============================] - 0s 63us/sample - loss: 0.3620 - acc: 0.86

In [10]:
print(bin_model.predict(pd.DataFrame(bin_X)))
print(reg_model.predict(pd.DataFrame(reg_X)))
print(multi_model.predict(pd.DataFrame(multi_X)))

[[0.14549029]
 [0.1633076 ]
 [0.16906926]
 [0.16610196]
 [0.14649823]
 [0.13029516]
 [0.15939015]
 [0.15105271]
 [0.2001062 ]
 [0.18559387]
 [0.1608339 ]
 [0.19708464]
 [0.21870443]
 [0.24911615]
 [0.17555109]
 [0.17335296]
 [0.20595607]
 [0.24297684]
 [0.21566164]
 [0.25859392]
 [0.27434713]
 [0.29519808]
 [0.35071594]
 [0.35252962]
 [0.38002574]
 [0.41304472]
 [0.41071752]
 [0.4100741 ]
 [0.43701547]
 [0.50502   ]
 [0.5256657 ]
 [0.48725232]
 [0.47653276]
 [0.47093663]
 [0.6052704 ]
 [0.6169761 ]
 [0.58137697]
 [0.6679027 ]
 [0.7304292 ]
 [0.7730628 ]
 [0.8189779 ]
 [0.8121382 ]
 [0.9006393 ]
 [0.8647927 ]
 [0.8977584 ]
 [0.87672883]
 [0.9387667 ]
 [0.8949224 ]
 [0.9393742 ]
 [0.9548084 ]
 [0.9320344 ]
 [0.9461826 ]
 [0.9395888 ]
 [0.9538585 ]
 [0.9368658 ]
 [0.95928454]
 [0.9627409 ]
 [0.966684  ]
 [0.9713676 ]
 [0.96731937]
 [0.9708972 ]
 [0.9735883 ]
 [0.9751626 ]
 [0.9749113 ]
 [0.9751367 ]
 [0.9766185 ]
 [0.97845805]
 [0.9835579 ]
 [0.14549029]
 [0.1633076 ]
 [0.16906926]
 [0.16

[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [11]:
FIXTURE_ROOT = '~/workspace/datarobot-user-models/tests/fixtures/drop_in_model_artifacts'
bin_model.save(os.path.expanduser(os.path.join(FIXTURE_ROOT, 'keras_bin.h5')))
reg_model.save(os.path.expanduser(os.path.join(FIXTURE_ROOT, 'keras_reg.h5')))
multi_model.save(os.path.expanduser(os.path.join(FIXTURE_ROOT, 'keras_multi.h5')))